In [1]:
# Preparation
import os
import sys
import shutil
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

sys.path.append('common')
sys.path.append('external')
import util
from domain_modules import *
from domain_models import *

import keras
import keras.backend as K
from keras.layers import Dense, Conv2D, MaxPooling2D, GlobalMaxPooling2D, Activation, Dropout, BatchNormalization, Flatten
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, Callback, TensorBoard

DATAROOT = os.path.join(os.path.expanduser('~'), '.kaggle/competitions/acoustic-scene-2018')

Using TensorFlow backend.


In [2]:
def pseudo_background_subtraction(X_train, X_valid, X_test):
    '''Subtract sample-wise mean = pseudo background subtraction'''
    X_train = np.array([X - np.mean(X, axis=1, keepdims=True) for X in X_train])
    X_valid = np.array([X - np.mean(X, axis=1, keepdims=True) for X in X_valid])
    X_test = np.array([X - np.mean(X, axis=1, keepdims=True) for X in X_test])
    return X_train, X_valid, X_test

d = Dataset(DATAROOT, X_modifier=pseudo_background_subtraction)

Applied special X modifier


In [3]:
K.clear_session()

model = model_cnn7_for_event_pattern(d.X_train.shape[1:], d.num_classes, lr=0.00007)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 40, 251, 64)       1408      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 251, 64)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 19, 251, 64)       256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 19, 251, 128)      57472     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 125, 128)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 125, 128)       512       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 125, 128)       82048     
__________

In [4]:
trainer = Trainer('event_pattern', d, model, 0.00007, 500, 32,
                  use_cyclic_lr=True, use_random_eraser=True, use_mixup=True)
trainer.fit()
after_fit(trainer, model)
fine_tune(trainer, model)

using cyclic lr
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 40, 251, 64)       1408      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 251, 64)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 19, 251, 64)       256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 19, 251, 128)      57472     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 125, 128)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 125, 128)       512       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 125, 128)       82048